# Tutorial: Generative QA with Retrieval Augmented Generation

In this tutorial, you'll learn how to run generative question answering by connecting a retriever to a generative LLM. You'll also learn how to use prompts with a generative model to tune your answers. The system should also generate a response like "Unanswerable" if no evidence is found.

You can plug-and-play this tutorial with most models on the HuggingFace model hub and also OpenAI LLMs. Some supported models include:
 - FLAN UL2-20B
 - FLAN T5 
 - Open AI ChatGPT (gpt-3.5-turbo)
 - InstructGPT(text-davinci-003)
 - lots more..

# Installing PrimeQA
First, we need to include the required modules.

In [ ]:
%%bash

pip install --upgrade pip
pip install primeqa

## Initialize the Retriever

### Pre-process your document collection here to be ready to be stored in your Neural Search Index.
In this step we download a publicly available .csv file from a Google Drive location and save it as .tsv.

In [1]:
# save your input document as a .tsv
import pandas as pd
url='https://drive.google.com/file/d/1LULJRPgN_hfuI2kG-wH4FUwXCCdDh9zh/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.to_csv('input.tsv', sep='\t')

### Initialize the model. In PrimeQA we use the SearchableCorpus class for searching through your corpus.

For DPR, you need to point to a question and context encoder models available via the HuggingFace model hub.

In [3]:
from primeqa.components.retriever.searchable_corpus import SearchableCorpus
retriever = SearchableCorpus(model_name="PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder", 
                              query_encoder_model_name_or_path="PrimeQA/XOR-TyDi_monolingual_DPR_qry_encoder", 
                              batch_size=64, top_k=10)

{"time":"2023-06-14 15:48:06,014", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-06-14 15:48:06,527", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


### Add your documents into the searchable corpus through PrimeQA's built-in pre-processor.

PrimeQA has a built-in class called DocumentCollection which pre-processes input.tsv to match the following format as needed by DPR:

`id \t text \t title_of_document`

Note: since DPR is based on an encoder language model the typical sequence length is 512 max sub-word tokens. So please make sure your documents are split into text length of ~220 words.

In [4]:
from primeqa.ir.util.corpus_reader import DocumentCollection
doc_collection = DocumentCollection("input.tsv")

retriever.add_documents(doc_collection.get_processed_collection())

75it [00:00, 55004.86it/s]


75


75it [00:00, 82673.53it/s]


75
{"time":"2023-06-14 15:48:43,747", "name": "primeqa.ir.dense.dpr_top.dpr.index_simple_corpus", "level": "INFO", "message": "wrote passages_1_of_1.json.gz.records in 0 seconds"}
{"time":"2023-06-14 15:48:43,748", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "building index, reading data from dpr_index_dir/passages_1_of_1.json.gz.records, writing to dpr_index_dir/index_1_of_1.faiss"}
{"time":"2023-06-14 15:48:43,799", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 0 passages"}
{"time":"2023-06-14 15:48:43,802", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "calling index.add with 75 vectors"}
{"time":"2023-06-14 15:48:43,805", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 75 passages"}
{"time":"2023-06-14 15:48:43,805", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "finished building index, writi

Some weights of the model checkpoint at PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder were not used when initializing DPRQuestionEncoder: ['ctx_encoder.bert_model.encoder.layer.7.attention.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.7.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.11.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.7.intermediate.dense.bias', 'ctx_encoder.bert_model.encoder.layer.9.attention.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.1.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.3.attention.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.5.attention.output.LayerNorm.weight', 'ctx_encoder.bert_model.encoder.layer.6.intermediate.dense.weight', 'ctx_encoder.bert_model.encoder.layer.5.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.7.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.4.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder

Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder and are newly initialized: ['bert_model.encoder.layer.6.attention.self.query.weight', 'bert_model.encoder.layer.3.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.1.intermediate.dense.weight', 'bert_model.encoder.layer.4.output.dense.bias', 'bert_model.encoder.layer.7.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.3.output.dense.weight', 'bert_model.encoder.layer.7.attention.self.query.bias', 'bert_model.encoder.layer.2.attention.output.dense.bias', 'bert_model.encoder.layer.10.attention.output.dense.weight', 'bert_model.encoder.layer.2.attention.self.value.bias', 'bert_model.encoder.layer.10.attention.self.value.bias', 'bert_model.encoder.layer.6.attention.self.key.bias', 'bert_model.encoder.layer.1.output.dense.bias', 'bert_model.encoder.layer.9.intermediate.dense.bias', 'bert_

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRContextEncoderTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.


{"time":"2023-06-14 15:48:45,199", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss, reading shards from dpr_index_dir"}
{"time":"2023-06-14 15:48:45,200", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_1_of_1.json.gz.records"}
{"time":"2023-06-14 15:48:45,202", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss with 1 shards."}


# Initialize the Reader 

In this step you can use a generative LLM which can be prompted. This reader can be any of the generative models available in the HuggingFace model hub or OpenAI models.

In [5]:
from primeqa.components.reader import GenerativeReader

reader = GenerativeReader(model_type='HuggingFace', model_name='google/flan-t5-xl')
# setup an OpenAI generative reader : we support gpt-3.5-turbo and text-davinci-003
# reader = GenerativeReader(model_type='OpenAI', model_name='gpt-3.5-turbo', api_key='')


# Setup the RAG pipeline

Attach a retriever to a generative LLM. You can then prompt it to answer questions.

In [6]:
from primeqa.pipelines import RAG
pipeline = RAG(retriever, reader)

# Start asking questions

We "run" the pipeline we just created and also attach a prompt.

In [7]:
import json

questions = ['When was Idaho split in two?' , 'Who was Danny Nozel']
prompt_prefix = "Answer the following question after looking at the text."
#answers=reader.predict(questions=questions,prefix=prompt_prefix)
answers = pipeline.run(questions, prefix=prompt_prefix)
print(json.dumps(answers, indent=4))

Token indices sequence length is longer than the specified maximum sequence length for this model (1552 > 512). Running this sequence through the model will result in indexing errors


{
    "0": {
        "question": "When was Idaho split in two?",
        "answers": [
            {
                "example_id": 0,
                "span_answer_text": "SVB) techniques. SVB is a spectrally efficient transmission technique, but is limited in the number of channels it can transmit. The bandwidth of an SVB signal is twice the bandwidth of the modulating (or \"baseband\") signal, since the upper and lower sidebands around the carrier frequency each have a bandwidth as wide as the highest modulating frequency. Although the bandwidth of an AM signal is narrower than one using frequency modulation (FM), it is twice as wide as single-sideband techniques; it thus may be viewed as spectrally inefficient. Within a frequency band, only half as many transmissions (or \"channels\") can thus be accommodated. For this reason analog television employs a variant of single-sideband (known as SVB) techniques.",
                "confidence_score": 1
            }
        ],
        "passa